## PF14

In [ ]:
using Revise, Genie, TreeTools, DCAUtils, JLD2, PyPlot, BenchmarkTools

eq_start_seq = read_fasta_alignment("../data_Anc/iidSample1_SparsePF14_bmDCA_fasta.txt", 0.9);
nat_msa = read_fasta_alignment("../data_Anc/PF0014_hamming1_Alya.fasta", 0.9);
  
import KitMSA: fasta2matrix, letter2num, num2letter, extract_params, read_par_BM, set_max_field_to_0, symmetrize_J

PATH_PAR = "../data_Anc/Parameters_conv_PF14Sabrina.dat"
htmp, Jtmp = read_par_BM(PATH_PAR);
#h = set_max_field_to_0(htmp);
#J = symmetrize_J(Jtmp);
#J_new = permutedims(J, [1,3,2,4]);
extrema(J)
sum(abs2,Jtmp)
sum(abs2, htmp)

h = htmp; J = permutedims(Jtmp, [1,3,2,4]);
no_cod_bias = Dict(x => (1/length(amino2cod[cod2amino[x]])) for x in keys(cod2amino));
        codon_usage = no_cod_bias;
q = 21; L =53; codon_usage = Genie.no_cod_bias;

N_trees = 1; tree_file = "../data_Anc/PF00014_tree_no_doubles_full"; 
rng = random_gens(N_trees+1); codon_net = Genie.create_nested_codon_dict();length_of_moves = Genie.create_length_dict(codon_net);
    temp = 1.0;    all_codons = vcat([amino2cod[i] for i in 1:20]...);push!(all_codons, "TAG");push!(all_codons, "TAA");push!(all_codons, "TGA");

chains = [Genie.Storage(Int.(eq_start_seq[:,400]), tree_file, rng[n]) for n in 1:N_trees+1];

a = chains[1].tree.root; mu = 6.5; p = 0.5;

@btime Genie.run_gibbs_sampling_tree!($(chains[1]), $(data(a).seq), $(data(a).DNA), $(h), $(J), 
                    $(codon_net), $(codon_usage), $(length_of_moves), $(temp), $(L))

@btime Genie.run_metropolis_indels_tree!($(chains[1]), $(data(a).seq), $(data(a).DNA), $(h), $(J), 
                    $(all_codons), $(codon_usage), $(temp), $(L)) 
                    
@btime Genie.assign_sequences!($(chains[1].tree.root), $(chains[1]), $(h), $(J), 
                    $(codon_net), $(all_codons), $(codon_usage), $(length_of_moves), $(temp), $(mu), $(p), $(L))

seq_site = rand(1:53); nucleo_site = rand(1:3);
@btime Genie.prob_cond_tree!($(chains[1]), $(data(a).seq), $(data(a).DNA), $(h), $(J), $(seq_site), $(nucleo_site), 
    $(codon_net), $(codon_usage), $(length_of_moves), $(temp), $(L))

### invece di definire ogni singola variabile storage, prova a farne una e poi a fare le altre uguali


## DBD 

In [ ]:
using Revise, Genie, TreeTools, DCAUtils, JLD2, PyPlot, BenchmarkTools

nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];
@load "../data_Genie/pars_dbd.jld2"; h = h_dbd; J = J_dbd;
@load "../data_Anc/eq_sample_DBD.jld2";

q = 21; L =76;

no_cod_bias = Dict(x => (1/length(amino2cod[cod2amino[x]])) for x in keys(cod2amino));
        codon_usage = no_cod_bias;
q = 21; L =53; codon_usage = Genie.no_cod_bias;

N_trees = 1; tree_file = "../data_Anc/star_tree_DBD_hom.newick"; 
rng = random_gens(N_trees+1); codon_net = Genie.create_nested_codon_dict();length_of_moves = Genie.create_length_dict(codon_net);
    temp = 1.0;    all_codons = vcat([amino2cod[i] for i in 1:20]...);push!(all_codons, "TAG");push!(all_codons, "TAA");push!(all_codons, "TGA");

@btime chains = [Genie.Storage(Int.(eq_msa_dbd[:,400]), tree_file, rng[n]) for n in 1:N_trees+1];

a = chains[1].tree.root; mu = 6.5; p = 0.5;

@btime Genie.run_gibbs_sampling_tree!($(chains[1]), $(data(a).seq), $(data(a).DNA), $(h), $(J), 
                    $(codon_net), $(codon_usage), $(length_of_moves), $(temp), $(L))

@btime Genie.run_metropolis_indels_tree!($(chains[1]), $(data(a).seq), $(data(a).DNA), $(h), $(J), 
                    $(all_codons), $(codon_usage), $(temp), $(L)) 
                    
@btime Genie.assign_sequences!($(chains[1].tree.root), $(chains[1]), $(h), $(J), 
                    $(codon_net), $(all_codons), $(codon_usage), $(length_of_moves), $(temp), $(mu), $(p), $(L))



seq_site = rand(1:53); nucleo_site = rand(1:3);
@btime Genie.prob_cond_tree!($(chains[1]), $(data(a).seq), $(data(a).DNA), $(h), $(J), $(seq_site), $(nucleo_site), 
    $(codon_net), $(codon_usage), $(length_of_moves), $(temp), $(L))

### invece di definire ogni singola variabile storage, prova a farne una e poi a fare le altre uguali
